In [ ]:
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
x= pd.read_csv(r'C:\Users\Avishkar\OneDrive\Documents\hw dataset\x_train_all.csv')
y= pd.read_csv(r'C:\Users\Avishkar\OneDrive\Documents\hw dataset\y_train_all.csv')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
classifier = LogisticRegression(max_iter=1000)

In [6]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
cv_scores = cross_val_score(classifier, X_train, y_train, cv=kf)

In [8]:
for i, score in enumerate(cv_scores, 1):
    print(f'Fold {i} Cross-Validation Accuracy: {score}')

Fold 1 Cross-Validation Accuracy: 0.9291237113402062
Fold 2 Cross-Validation Accuracy: 0.9239690721649485
Fold 3 Cross-Validation Accuracy: 0.9303225806451613
Fold 4 Cross-Validation Accuracy: 0.9148387096774193
Fold 5 Cross-Validation Accuracy: 0.9212903225806451
Fold 6 Cross-Validation Accuracy: 0.9509677419354838
Fold 7 Cross-Validation Accuracy: 0.9393548387096774
Fold 8 Cross-Validation Accuracy: 0.9161290322580645
Fold 9 Cross-Validation Accuracy: 0.9251612903225807
Fold 10 Cross-Validation Accuracy: 0.9148387096774193


In [9]:
print(f'Average Cross-Validation Accuracy: {np.mean(cv_scores)}')

Average Cross-Validation Accuracy: 0.9265996009311606


In [10]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [11]:
y_pred_test = classifier.predict(X_test)

In [12]:
test_accuracy = accuracy_score(y_test, y_pred_test)

In [13]:
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.9293085655314758


### Hyperparameter optimization

#### VARIOUS PARAMETERS OF NEURAL NETWORK

In [53]:


params = {
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "activation_functions": ['logistic', 'tanh', 'relu'],
    "max_epochs": [50, 100, 200],
    "validation_thresholds": [0.1, 0.01, 0.001],
    "momentum_values": [0.9, 0.7, 0.5],
    "hidden_layer_sizes_list": [(100,), (50, 50), (100, 50, 25)]
}


In [45]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.neural_network import MLPClassifier

In [56]:
for hidden_layer_sizes in params["hidden_layer_sizes_list"]:
    for activation in params["activation_functions"]:
        for learning_rate in params["learning_rate"]:
            for momentum in params["momentum_values"]:
                for max_epoch in params["max_epochs"]:
                    for validation_threshold in params["validation_thresholds"]:
                        classifier = MLPClassifier(
                            hidden_layer_sizes=hidden_layer_sizes,
                            activation=activation,
                            learning_rate_init=learning_rate,
                            momentum=momentum,
                            max_iter=max_epoch,
                            validation_fraction=validation_threshold,
                            random_state=42)

In [57]:
print(f'Parameters: Hidden Layer Sizes={hidden_layer_sizes}, Activation={activation}, '
                              f'Learning Rate={learning_rate}, Momentum={momentum}, Epochs={max_epoch}, '
                              f'Validation Threshold={validation_threshold}')

Parameters: Hidden Layer Sizes=(100, 50, 25), Activation=relu, Learning Rate=0.3, Momentum=0.5, Epochs=200, Validation Threshold=0.001


#### shaping the numpy array 

In [69]:
x_array = x.values

In [70]:
x_reshaped = x_array.reshape(-1, 48, 48, 1)

In [72]:
image_height, image_width, num_channels = x_reshaped.shape[1], x_reshaped.shape[2], x_reshaped.shape[3]

num_classes = len(np.unique(y))
y_one_hot = tf.keras.utils.to_categorical(y, num_classes)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(x_reshaped, y_one_hot, test_size=0.2, random_state=42)

#### hyperparameters for grid search

In [78]:
param_grid = {
    'filters': [32, 64],
    'kernel_size': [(3, 3), (5, 5)],
    'pool_size': [(2, 2), (3, 3)],
    'dense_neurons': [128, 256],
    'dropout_rate': [0.25, 0.5],
    'learning_rate': [0.001, 0.01],
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

In [79]:
best_accuracy = 0
best_params = None

In [ ]:
for filters in param_grid['filters']:
    for kernel_size in param_grid['kernel_size']:
        for pool_size in param_grid['pool_size']:
            for dense_neurons in param_grid['dense_neurons']:
                for dropout_rate in param_grid['dropout_rate']:
                    for learning_rate in param_grid['learning_rate']:
                        for batch_size in param_grid['batch_size']:
                            for epochs in param_grid['epochs']:
                           
                                model = Sequential()
                                model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=(image_height, image_width, num_channels)))
                                model.add(MaxPooling2D(pool_size=pool_size))
                                model.add(Flatten())
                                model.add(Dense(dense_neurons, activation='relu'))
                                model.add(Dropout(dropout_rate))
                                model.add(Dense(num_classes, activation='softmax'))

                                
                                optimizer = Adam(learning_rate=learning_rate)
                                model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

                            
                                datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
                                train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)

                    
                                model.fit(train_generator, epochs=epochs, validation_data=(X_test, y_test), batch_size=batch_size, verbose=0)

                           
                                _, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

                              
                                print(f'Parameters: Filters={filters}, Kernel Size={kernel_size}, Pool Size={pool_size}, Dense Neurons={dense_neurons}, Dropout Rate={dropout_rate}, Learning Rate={learning_rate}, Batch Size={batch_size}, Epochs={epochs}')
                                print(f'Test Accuracy: {test_accuracy}\n')

                                
                                if test_accuracy > best_accuracy:
                                    best_accuracy = test_accuracy
                                    best_params = {
                                        'filters': filters,
                                        'kernel_size': kernel_size,
                                        'pool_size': pool_size,
                                        'dense_neurons': dense_neurons,
                                        'dropout_rate': dropout_rate,
                                        'learning_rate': learning_rate,
                                        'batch_size': batch_size,
                                        'epochs': epochs
                                    }

Parameters: Filters=32, Kernel Size=(3, 3), Pool Size=(2, 2), Dense Neurons=128, Dropout Rate=0.25, Learning Rate=0.001, Batch Size=32, Epochs=10
Test Accuracy: 0.8787409663200378

Parameters: Filters=32, Kernel Size=(3, 3), Pool Size=(2, 2), Dense Neurons=128, Dropout Rate=0.25, Learning Rate=0.001, Batch Size=32, Epochs=20
Test Accuracy: 0.7770897746086121

Parameters: Filters=32, Kernel Size=(3, 3), Pool Size=(2, 2), Dense Neurons=128, Dropout Rate=0.25, Learning Rate=0.001, Batch Size=64, Epochs=10
Test Accuracy: 0.8276574015617371

Parameters: Filters=32, Kernel Size=(3, 3), Pool Size=(2, 2), Dense Neurons=128, Dropout Rate=0.25, Learning Rate=0.001, Batch Size=64, Epochs=20
Test Accuracy: 0.7858617305755615

Parameters: Filters=32, Kernel Size=(3, 3), Pool Size=(2, 2), Dense Neurons=128, Dropout Rate=0.25, Learning Rate=0.01, Batch Size=32, Epochs=10
Test Accuracy: 0.23581011593341827

Parameters: Filters=32, Kernel Size=(3, 3), Pool Size=(2, 2), Dense Neurons=128, Dropout Rate=0